Берем задачку с отзывами и экспериментируем:
- учим деревья и сравниваем с лог регом
- добавляем признаки по числу слов (вообще посмотреть, может можно сделать классификатор - словарик)
- учим тематическую модель (подобрать число тем, поиграть с параметрами)

In [145]:
import pandas as pd
import numpy as np
from collections import Counter
from nltk import word_tokenize
import emoji
from string import ascii_letters, punctuation, digits
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
import pymorphy2
import gensim
from stop_words import get_stop_words
import re

In [2]:
data = pd.read_excel('../data/summer_reviews.xls')

In [3]:
data.columns = ['rating', 'content', 'date']

Для начала посмотрим какие вообще символы встречаются в отзывах.

In [4]:
content_symbols = Counter()
_ = data.content.apply(lambda x: content_symbols.update(str(x)))

Можно увидеть, что встречаются эмоджи, небуквенные символы, символы с диакритическими знаками, а также иероглифы.

In [5]:
content_symbols.most_common()[140:160]

[('q', 26),
 ('👎', 26),
 ('Q', 25),
 ('😉', 23),
 ('☆', 21),
 ('Ё', 19),
 ('😃', 19),
 ('j', 17),
 ('👏', 17),
 ('—', 16),
 ('\u200b', 16),
 ('😄', 16),
 ('☺', 15),
 ('=', 15),
 ('B', 15),
 ('V', 13),
 ('Щ', 13),
 ('😂', 12),
 ('J', 11),
 ('💪', 11)]

In [6]:
data['tokens'] = data.content.apply(lambda x: word_tokenize(str(x)))

In [7]:
def count_emojis(tokens):
    return len([token for token in tokens if token in emoji.UNICODE_EMOJI])

In [8]:
data['emoji_num'] = data.tokens.apply(lambda x: count_emojis(x))

In [9]:
cyrillic_letters = set([chr(i) for i in range(ord('а'), ord('я') + 1)] +
                       [chr(i) for i in range(ord('А'), ord('Я') + 1)] +
                       ['ё', 'Ё'])

In [10]:
def count_cyrillic(tokens):
    count = 0
    for token in tokens:
        for ch in token:
            if ch in cyrillic_letters:
                count += 1
    return count

In [11]:
def count_latin(tokens):
    count = 0
    for token in tokens:
        for ch in token:
            if ch in ascii_letters:
                count += 1
    return count

In [12]:
data['cyrillic_num'] = data.tokens.apply(lambda x: count_cyrillic(x))

In [13]:
data['latin_num'] = data.tokens.apply(lambda x: count_latin(x))

In [14]:
# дополним пунктуацию
punctuation = set(punctuation).union((' ', '«', '»', '—', '–', '“', '”', '…'))

In [15]:
other_non_alpha = [sym for sym in content_symbols if (not sym.isalpha()
                                                      and sym not in ascii_letters
                                                      and sym not in cyrillic_letters
                                                      and sym not in punctuation
                                                      and sym not in digits
                                                      and sym not in emoji.UNICODE_EMOJI)]

Остальные символы будем отбрасывать.

In [16]:
data.tokens = data.tokens.apply(lambda x: [token for token in x if token not in other_non_alpha])

In [17]:
def count_other(tokens):
    count = 0
    for token in tokens:
        for ch in token:
            if (ch.isalpha()
                and ch not in ascii_letters
                and ch not in cyrillic_letters):
                count += 1
    return count

In [18]:
# количество букв из других алфавитов, косвенно указывающее на другой язык
data['other_num'] = data.tokens.apply(lambda x: count_other(str(x)))

In [19]:
def count_words(tokens):
    return len([token for token in tokens if token.isalpha()])

In [20]:
def count_uppercase(tokens):
    count = 0
    for token in tokens:
        for ch in token:
            if (ch.isalpha
                and ch.isupper()):
                count += 1
    return count

In [21]:
def count_uppercase_words(tokens):
    return len([token for token in tokens if token.isalpha() and token.isupper() and len(token) > 1])

In [22]:
data['word_num'] = data.tokens.apply(lambda x: count_words(x))

In [23]:
data['uppercase_num'] = data.tokens.apply(lambda x: count_uppercase(x))

In [24]:
data['uppercase_word_num'] = data.tokens.apply(lambda x: count_uppercase_words(x))

In [25]:
data['target'] = (data.rating > 3).astype(int)

Подготовим отдельные токены для векторизации. Попробуем включить в токены также и эмоджи.

In [28]:
morph = pymorphy2.MorphAnalyzer(lang='ru')

In [29]:
def lemmatize(token, morph=morph):
    return morph.parse(token)[0].normal_form

In [30]:
def process_tokens(tokens):
    result = []
    for token in tokens:
        if (token.isalpha()):
            result.append(lemmatize(token).lower())
        elif token in emoji.UNICODE_EMOJI:
            result.append(token)
        else:
            pass
    return result

In [31]:
data['vec_tokens'] = data.tokens.apply(lambda x: ' '.join(process_tokens(x)))

In [32]:
X_train, X_test, y_train, y_test = train_test_split(data.drop('target', axis=1), data['target'], test_size=0.2, random_state=42)

In [33]:
vec = TfidfVectorizer(min_df=0.001, max_df=0.999)
vec.fit(X_train.vec_tokens.values)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=0.999,
                max_features=None, min_df=0.001, ngram_range=(1, 1), norm='l2',
                preprocessor=None, smooth_idf=True, stop_words=None,
                strip_accents=None, sublinear_tf=False,
                token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
                vocabulary=None)

In [34]:
X_train_vec = vec.transform(X_train.vec_tokens.values)

In [35]:
X_test_vec = vec.transform(X_test.vec_tokens.values)

In [36]:
vec_vocab = {v:k for k, v in vec.vocabulary_.items()}

In [37]:
lr = LogisticRegression()

In [38]:
lr.fit(X_train_vec, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [39]:
lr_preds = lr.predict(X_test_vec)

Качество растет, когда почти не урезаем по частотам.

In [40]:
f1_score(y_test, lr_preds)

0.9437268002969562

Предположение о том, что эмоджи будут полезны не подтвердилось.

In [41]:
for i in lr.coef_[0].argsort()[::-1][:10]:
    print(vec_vocab[i])

удобно
отличный
спасибо
удобный
супер
хорошо
отлично
нравиться
хороший
довольный


In [42]:
for i in lr.coef_[0].argsort()[:10]:
    print(vec_vocab[i])

не
невозможно
ужасно
ужасный
антивирус
вылетать
отвратительный
прошивка
рута
root


Составим словари оценочных слов.

In [48]:
neg_dict = set([vec_vocab[i] for i in lr.coef_[0].argsort()[:100]])
pos_dict = set([vec_vocab[i] for i in lr.coef_[0].argsort()[::-1][:100]])

In [49]:
rf = RandomForestClassifier()

In [50]:
rf.fit(X_train_vec, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [51]:
rf_preds = rf.predict(X_test_vec)

In [52]:
f1_score(y_test, rf_preds)

0.9385491606714628

Попробуем использовать фичи текста.

In [53]:
feat_cols = data.columns.drop(['rating', 'content', 'date', 'tokens', 'target', 'vec_tokens'])

In [54]:
feat_cols

Index(['emoji_num', 'cyrillic_num', 'latin_num', 'other_num', 'word_num',
       'uppercase_num', 'uppercase_word_num'],
      dtype='object')

In [55]:
X_train_feats = X_train[feat_cols]
X_test_feats = X_test[feat_cols]

In [56]:
lr.fit(X_train_feats, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [57]:
lr_preds = lr.predict(X_test_feats)

In [58]:
f1_score(y_test, lr_preds)

0.9060139860139861

In [59]:
rf.fit(X_train_vec, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [60]:
rf_preds = rf.predict(X_test_vec)

In [61]:
f1_score(y_test, rf_preds)

0.938885560215698

В целом результат достаточно неплохой.

Попробуем все вместе.

In [62]:
X_train_vec.todense().shape

(16527, 768)

In [63]:
X_train_all = np.hstack([X_train_vec.todense(), X_train_feats])
X_test_all = np.hstack([X_test_vec.todense(), X_test_feats])

In [64]:
lr = LogisticRegression(max_iter=1000)

In [65]:
lr.fit(X_train_all, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [66]:
lr_preds = lr.predict(X_test_all)

In [67]:
f1_score(y_test, lr_preds)

0.94351371386212

In [68]:
rf.fit(X_train_all, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [69]:
rf_preds = rf.predict(X_test_all)

In [70]:
f1_score(y_test, rf_preds)

0.9399641577060931

Особой пользы это не дает.

Попробуем воспользоваться словарями.

In [76]:
data['pos_num'] = data.tokens.apply(lambda x: len([word for word in x if word.lower() in pos_dict]))
data['neg_num'] = data.tokens.apply(lambda x: len([word for word in x if word.lower() in neg_dict]))

In [83]:
X_train, X_test, y_train, y_test = train_test_split(data.drop('target', axis=1), data['target'], test_size=0.2, random_state=42)

In [84]:
X_test_dict = X_test[['pos_num', 'neg_num']]

Таким примитивным образом тоже можно получить относительно неплохой результат.

In [92]:
f1_score(y_test, X_test_dict.pos_num - X_test_dict.neg_num > 0)

0.7508746087276744

In [93]:
X_train_dict = X_train[['pos_num', 'neg_num']]

In [94]:
lr.fit(X_train_dict, y_train)
lr_preds = lr.predict(X_test_dict)

In [95]:
f1_score(y_test, lr_preds)

0.9234980550352975

In [126]:
stop_words = get_stop_words('ru')

In [151]:
def combine_negation(text):
    return re.sub(r'([Нн][Ее]) (\w)', '\g<1>_\g<2>', text)

In [152]:
data['lda_tokens'] = data.vec_tokens.apply(lambda x: combine_negation(x))
data['lda_tokens'] = data.lda_tokens.apply(lambda x: x.split())
data['lda_tokens'] = data.lda_tokens.apply(lambda x: [token for token in x if token not in stop_words and token not in emoji.UNICODE_EMOJI])

In [161]:
data.loc[data.vec_tokens.str.contains('не'), 'lda_tokens']

3        [стать, зависать, работа, антивирус, далёкий, ...
11                       [нормально, уведомление, удалять]
12       [не_стартовать, доступ, gps, sms, звонок, адре...
13           [удобно, работать, замечательный, подвисания]
25       [заход, дважды, требовать, оплата, электроэнер...
                               ...                        
20648    [фига, писать, разраб, читать, не_уметь, невос...
20649    [писать, рута, доступ, обнова, работать, отлич...
20651    [новый, приложение, отсутствовать, опция, упра...
20652    [meizu, обновление, писать, тело, рутованный, ...
20654    [шляпа, роот, право, бесполезный, прога, разра...
Name: lda_tokens, Length: 5260, dtype: object

In [168]:
dictionary = gensim.corpora.Dictionary(data.lda_tokens)
dictionary.filter_extremes(no_below=5, no_above=0.4, keep_n=100000)
bow_corpus = [dictionary.doc2bow(doc) for doc in data.lda_tokens.tolist()]
tfidf = gensim.models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

In [173]:
lda_model = gensim.models.LdaMulticore(corpus_tfidf, num_topics=4, id2word=dictionary, passes=3, workers=4)

In [174]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.160*"удобно" + 0.033*"быстро" + 0.032*"работать" + 0.015*"пользоваться" + 0.015*"молодец" + 0.015*"круто" + 0.012*"приложение" + 0.010*"не_мочь" + 0.008*"проблема" + 0.008*"обновление"
Topic: 1 
Words: 0.093*"нравиться" + 0.092*"супер" + 0.079*"отличный" + 0.056*"устраивать" + 0.045*"приложение" + 0.025*"удобно" + 0.023*"довольный" + 0.022*"понятно" + 0.013*"понравиться" + 0.012*"работать"
Topic: 2 
Words: 0.085*"отлично" + 0.018*"класс" + 0.018*"работать" + 0.012*"приложение" + 0.012*"пароль" + 0.009*"перевод" + 0.008*"вводить" + 0.008*"карта" + 0.008*"вход" + 0.008*"обновление"
Topic: 3 
Words: 0.077*"удобный" + 0.068*"приложение" + 0.059*"хороший" + 0.033*"норма" + 0.018*"нормально" + 0.010*"большой" + 0.009*"антивирус" + 0.009*"банк" + 0.008*"прошивка" + 0.008*"программа"
